In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
cd drive/MyDrive/Colab\ Notebooks/scripts

/content/drive/MyDrive/Colab Notebooks/scripts


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import os

In [ ]:
from feature_selection import forwardFeatureSelection

from NonLinCFA import NonLinCFA

from aux_NonLinCFA import standardize,unfold_dataset,compute_r2,prepare_target,prepare_features,aggregate_unfolded_data,aggregate_unfolded_data_onlyTrain,FS_with_linearWrapper,compare_methods, compute_r2, aggregate_data_withoutUnfolding

In [ ]:
cd ..

/content/drive/MyDrive/Colab Notebooks


# NonLinCFA aggregations

## Temp+Prec

In [ ]:
cd ..

/content/drive/MyDrive/Colab Notebooks


In [ ]:
### BOTH
#basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
basins = ['Garda_Mincio', 'Emiliani1']

path_target = "./csv/"
path_features = './features_allvalues/'
destination_folder = './NonLinCFA/temp_prec_inverted_for/top_right/'
plots_folder = './NonLinCFA/for_plots/inverted_for/top_right/'

for basin in basins:
    selected_colnames_CMI5 = []
    outputs = []
    print('####################' + basin + '####################')

    target_df_train, target_df_val, target_df_test, target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01',
                                                                                     max_test='2020-01-01', path=path_target+basin+'.csv',
                                                                                     window_size = 1)

    eps = 0.001
    actual_path = path_features+basin+'_aggreg.csv'

    output, aggregate_trainVal, aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                       cols_order='top_right')
    outputs.append(output)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),
                                                      np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    #if len(res['selectedFeatures']) <= 3:
    #  res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),
    #                                                  np.array(target_df_trainVal.mean_std),res,6,1)
    #  selectedFeatures='selectedFeatures'
    #  print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_best5_CMI_train.csv'
    val_string = destination_folder + basin + '_nonLinCFA_best5_CMI_val.csv'
    test_string = destination_folder + basin + '_nonLinCFA_best5_CMI_test.csv'

    X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
    X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
    X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
    X_test_CMI5 = aggregate_test.loc[:,selected_colnames[0:5]]

    selected_colnames_CMI5.append(aggregate_trainVal.loc[:,selected_colnames[0:5]].columns.values)

    X_train_CMI5.to_csv(train_string, index=False)
    X_validation_CMI5.to_csv(val_string, index=False)
    X_test_CMI5.to_csv(test_string, index=False)

    print('###### Regression ######')
    lin_regr = LinearRegression()

    # CMI best 5
    lin_regr.fit(X_train_validation_CMI5, target_df_trainVal.mean_std)
    print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(X_train_validation_CMI5, target_df_trainVal.mean_std),3))
    print("Test R2 linear regression CMI best 5: ", round(lin_regr.score(X_test_CMI5, target_df_test.mean_std),3), "\n")

    print('###### Binary Classification ######')

    target_df_train = target_df_train.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_val = target_df_val.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_test = target_df_test.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_trainVal = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)

    log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

    # CMI best 5
    log_regr.fit(X_train_validation_CMI5, target_df_trainVal)
    print("Train accuracy logistic regression CMI best 5: ", round(log_regr.score(X_train_validation_CMI5, target_df_trainVal),3))
    print("Test accuracy logistic regression CMI best 5: ", round(log_regr.score(X_test_CMI5, target_df_test),3), "\n")

    output_string = plots_folder + basin + '_aggregations.npy'
    sel_col_string = plots_folder + basin + '_chosen_features.npy'
    np.save(sel_col_string, selected_colnames_CMI5)
    np.save(output_string, outputs)

####################Garda_Mincio####################
Number of features: 67

Number of aggregated features: 1

Number of features: 67

Number of aggregated features: 1

Number of features: 67

Number of aggregated features: 1

Number of features: 67

Number of aggregated features: 5

Number of features: 67

Number of aggregated features: 5

Number of features: 67

Number of aggregated features: 3

Number of features: 67

Number of aggregated features: 2

Number of features: 67

Number of aggregated features: 4

Number of features: 67

Number of aggregated features: 6

Number of features: 67

Number of aggregated features: 4

Number of features: 67

Number of aggregated features: 6

Number of features: 67

Number of aggregated features: 5

Number of features: 67

Number of aggregated features: 3

Number of features: 67

Number of aggregated features: 4

----- MI Scores -----
[(30, 0.12349633338976866), (29, 0.10682474937409657), (34, 0.10425738764951727), (48, 0.10303196831763571), (36,

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


####################Emiliani1####################
Number of features: 172

Number of aggregated features: 10

Number of features: 172

Number of aggregated features: 9

Number of features: 172

Number of aggregated features: 9

Number of features: 172

Number of aggregated features: 11

Number of features: 172

Number of aggregated features: 12

Number of features: 172

Number of aggregated features: 8

Number of features: 172

Number of aggregated features: 6

Number of features: 172

Number of aggregated features: 6

Number of features: 172

Number of aggregated features: 6

Number of features: 172

Number of aggregated features: 4

Number of features: 172

Number of aggregated features: 9

Number of features: 172

Number of aggregated features: 8

Number of features: 172

Number of aggregated features: 3

Number of features: 172

Number of aggregated features: 6

----- MI Scores -----
[(79, 0.17873172971596582), (71, 0.15473962339335762), (85, 0.15444996467906918), (87, 0.1538137691

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [ ]:
### BOTH
#basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
basins = ['Emiliani1']

path_target = "./csv/"
path_features = './features_allvalues/'
destination_folder = './NonLinCFA/temp_prec_inverted_for/bottom_right/'
plots_folder = './NonLinCFA/for_plots/inverted_for/bottom_right/'

for basin in basins:
    selected_colnames_CMI5 = []
    outputs = []
    print('####################' + basin + '####################')

    target_df_train, target_df_val, target_df_test, target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01',
                                                                                     max_test='2020-01-01', path=path_target+basin+'.csv',
                                                                                     window_size = 1)

    eps = 0.001
    actual_path = path_features+basin+'_aggreg.csv'

    output, aggregate_trainVal, aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                       cols_order='bottom_right')
    #aggregate_trainVal_string = plots_folder + basin + '_trainVal_aggreg'
    #aggregate_test_string = plots_folder + basin + '_test_aggreg'
    #aggregate_trainVal.to_csv(aggregate_trainVal_string, index = False)
    #aggregate_test.to_csv(aggregate_test_string, index = False)
    outputs.append(output)

    #selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    #print('\nFull model and selected features with wrapper\n')
    #compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    #print('\nFull model and best 5 selected features with wrapper\n')
    #compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    #train_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_train.csv'
    #val_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_val.csv'
    #test_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_test.csv'

    #X_train_wrapper = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
    #X_validation_wrapper = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
    #X_train_validation_wrapper = pd.concat([X_train_wrapper, X_validation_wrapper])
    #X_test_wrapper = aggregate_test.loc[:,selected_colnames[0:5]]

    #X_train_wrapper.to_csv(train_string, index=False)
    #X_validation_wrapper.to_csv(val_string, index=False)
    #X_test_wrapper.to_csv(test_string, index=False)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),
                                                      np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    #if len(res['selectedFeatures']) <= 3:
    #  res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),
    #                                                  np.array(target_df_trainVal.mean_std),res,6,1)
    #  selectedFeatures='selectedFeatures'
    #  print(f'\n{res[selectedFeatures]}\n')

    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_best5_CMI_train.csv'
    val_string = destination_folder + basin + '_nonLinCFA_best5_CMI_val.csv'
    test_string = destination_folder + basin + '_nonLinCFA_best5_CMI_test.csv'

    X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
    X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
    X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
    X_test_CMI5 = aggregate_test.loc[:,selected_colnames[0:5]]

    selected_colnames_CMI5.append(aggregate_trainVal.loc[:,selected_colnames[0:5]].columns.values)

    X_train_CMI5.to_csv(train_string, index=False)
    X_validation_CMI5.to_csv(val_string, index=False)
    X_test_CMI5.to_csv(test_string, index=False)

    train_string = destination_folder + basin + '_nonLinCFA_CMI_train.csv'
    val_string = destination_folder + basin + '_nonLinCFA_CMI_val.csv'
    test_string = destination_folder + basin + '_nonLinCFA_CMI_test.csv'

    X_train_CMI = aggregate_trainVal.loc[:410,selected_colnames]
    X_validation_CMI = aggregate_trainVal.loc[411:,selected_colnames]
    X_train_validation_CMI = pd.concat([X_train_CMI, X_validation_CMI])
    X_test_CMI = aggregate_test.loc[:,selected_colnames]

    #X_train_CMI.to_csv(train_string, index=False)
    #X_validation_CMI.to_csv(val_string, index=False)
    #X_test_CMI.to_csv(test_string, index=False)

    print('###### Regression ######')
    lin_regr = LinearRegression()

    # CMI
    #lin_regr.fit(X_train_validation_CMI, target_df_trainVal.mean_std)
    #print("Train R2 linear regression CMI: ", round(lin_regr.score(X_train_validation_CMI, target_df_trainVal.mean_std),3))
    #print("Test R2 linear regression CMI: ", round(lin_regr.score(X_test_CMI, target_df_test.mean_std),3), "\n")

    # CMI best 5
    lin_regr.fit(X_train_validation_CMI5, target_df_trainVal.mean_std)
    print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(X_train_validation_CMI5, target_df_trainVal.mean_std),3))
    print("Test R2 linear regression CMI best 5: ", round(lin_regr.score(X_test_CMI5, target_df_test.mean_std),3), "\n")

    # wrapper
    #lin_regr.fit(X_train_validation_wrapper, target_df_trainVal.mean_std)
    #print("Train R2 linear regression wrapper: ", round(lin_regr.score(X_train_validation_wrapper, target_df_trainVal.mean_std),3))
    #print("Test R2 linear regression wrapper: ", round(lin_regr.score(X_test_wrapper, target_df_test.mean_std),3), "\n")

    print('###### Binary Classification ######')

    target_df_train = target_df_train.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_val = target_df_val.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_test = target_df_test.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_trainVal = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)

    log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

    # CMI
    #log_regr.fit(X_train_validation_CMI, target_df_trainVal)
    #print("Train accuracy logistic regression CMI: ", round(log_regr.score(X_train_validation_CMI, target_df_trainVal),3))
    #print("Test accuracy logistic regression CMI: ", round(log_regr.score(X_test_CMI, target_df_test),3), "\n")

    # CMI best 5
    log_regr.fit(X_train_validation_CMI5, target_df_trainVal)
    print("Train accuracy logistic regression CMI best 5: ", round(log_regr.score(X_train_validation_CMI5, target_df_trainVal),3))
    print("Test accuracy logistic regression CMI best 5: ", round(log_regr.score(X_test_CMI5, target_df_test),3), "\n")

    # wrapper
    #log_regr.fit(X_train_validation_wrapper, target_df_trainVal)
    #print("Train accuracy logistic regression wrapper: ", round(log_regr.score(X_train_validation_wrapper, target_df_trainVal),3))
    #print("Test accuracy logistic regression wrapper: ", round(log_regr.score(X_test_wrapper, target_df_test),3), "\n")

    output_string = plots_folder + basin + '_aggregations.npy'
    sel_col_string = plots_folder + basin + '_chosen_features.npy'
    np.save(sel_col_string, selected_colnames_CMI5)
    np.save(output_string, outputs)

####################Emiliani1####################
Number of features: 172

Number of aggregated features: 9

Number of features: 172

Number of aggregated features: 11

Number of features: 172

Number of aggregated features: 13

Number of features: 172

Number of aggregated features: 11

Number of features: 172

Number of aggregated features: 9

Number of features: 172

Number of aggregated features: 6

Number of features: 172

Number of aggregated features: 4

Number of features: 172

Number of aggregated features: 5

Number of features: 172

Number of aggregated features: 6

Number of features: 172

Number of aggregated features: 6

Number of features: 172

Number of aggregated features: 4

Number of features: 172

Number of aggregated features: 2

Number of features: 172

Number of aggregated features: 2

Number of features: 172

Number of aggregated features: 5

----- MI Scores -----
[(74, 0.17384914834104392), (68, 0.15726368355257056), (80, 0.15724934549989203), (84, 0.1492191981

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


## Temp + Prec + Snow

In [ ]:
# basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']

In [ ]:
basin = 'Adda'
path_target = "./csv/"
eps = 0.01
path_features_snow = './features_allvalues/snow/copernicus/'
snow_actual_path = path_features_snow+basin+'_aggreg_sd_allCoord.csv'

In [ ]:
target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')

output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS',
                                                                          'cyclostationary_mean_HS_1w',
                                                                          'cyclostationary_mean_HS_4w',
                                                                          'cyclostationary_mean_HS_8w',
                                                                          'cyclostationary_mean_HS_12w',
                                                                          'cyclostationary_mean_HS_16w',
                                                                          'cyclostationary_mean_HS_24w'
                                                                        ],
                                                                    target_df_trainVal, eps=eps,
                                                                    max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                                  aggreg_small_coord_groups = True, scale = 0.26)

In [ ]:
basins = ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino']
path_target = "./csv/"
path_features_snow = './features_allvalues/snow/copernicus/relevant_coords/'
path_features='./features_allvalues/'

destination_folder = './NonLinCFA/temp_prec_snow_copernicus/relevant_coords/'


for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                                     path=path_target+basin+'.csv')

    eps = 0.01
    actual_path = path_features+basin+'_aggreg.csv'
    snow_actual_path = path_features_snow+basin+'_aggreg_sd_allCoord.csv'


    output, aggregate_trainVal, aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
    if os.path.isfile(snow_actual_path):
      output_snow, aggregate_trainVal_snow, aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS',
                                                                              'cyclostationary_mean_HS_1w',
                                                                              'cyclostationary_mean_HS_4w',
                                                                              'cyclostationary_mean_HS_8w',
                                                                              'cyclostationary_mean_HS_12w',
                                                                              'cyclostationary_mean_HS_16w',
                                                                              'cyclostationary_mean_HS_24w'
                                                                              ],
                                                                        target_df_trainVal, eps=eps,
                                                                        max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                                    aggreg_small_coord_groups = True, scale = 0.26)


      aggregate_trainVal = pd.concat((aggregate_trainVal_snow, aggregate_trainVal),axis=1)
      aggregate_test = pd.concat((aggregate_test_snow, aggregate_test),axis=1)

    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_train_withSnow.csv'
    val_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_val_withSnow.csv'
    test_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_test_withSnow.csv'

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_best5_CMI_train_withSnow.csv'
    val_string = destination_folder + basin + '_nonLinCFA_best5_CMI_val_withSnow.csv'
    test_string = destination_folder + basin + '_nonLinCFA_best5_CMI_test_withSnow.csv'

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    train_string = destination_folder + basin + '_nonLinCFA_CMI_train_withSnow.csv'
    val_string = destination_folder + basin + '_nonLinCFA_CMI_val_withSnow.csv'
    test_string = destination_folder + basin + '_nonLinCFA_CMI_test_withSnow.csv'

    aggregate_trainVal.loc[:410,selected_colnames].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames].to_csv(test_string, index=False)


####################Adda####################
Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 4

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 5

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 4

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 1

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 3

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 1

Number of features: 92

e

In [ ]:
basins = ['Emiliani1', 'Emiliani2', 'Garda_Mincio']
path_target = "./csv/"
path_features_snow = './features_allvalues/snow/copernicus/relevant_coords/'
path_features='./features_allvalues/'

destination_folder = './NonLinCFA/temp_prec_snow_copernicus/relevant_coords/'


for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                                     path=path_target+basin+'.csv')

    eps = 0.1
    actual_path = path_features+basin+'_aggreg.csv'
    snow_actual_path = path_features_snow+basin+'_aggreg_sd_allCoord.csv'


    output, aggregate_trainVal, aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
    if os.path.isfile(snow_actual_path):
      output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS',
                                                                              'cyclostationary_mean_HS_1w',
                                                                              'cyclostationary_mean_HS_4w',
                                                                              'cyclostationary_mean_HS_8w',
                                                                              'cyclostationary_mean_HS_12w',
                                                                              'cyclostationary_mean_HS_16w',
                                                                              'cyclostationary_mean_HS_24w'
                                                                             ],
                                                                         target_df_trainVal, eps=eps,
                                                                         max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                                    aggreg_small_coord_groups = True, scale = 0.26)


      aggregate_trainVal = pd.concat((aggregate_trainVal_snow, aggregate_trainVal),axis=1)
      aggregate_test = pd.concat((aggregate_test_snow, aggregate_test),axis=1)

    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_train_withSnow.csv'
    val_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_val_withSnow.csv'
    test_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_test_withSnow.csv'

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_best5_CMI_train_withSnow.csv'
    val_string = destination_folder + basin + '_nonLinCFA_best5_CMI_val_withSnow.csv'
    test_string = destination_folder + basin + '_nonLinCFA_best5_CMI_test_withSnow.csv'

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

####################Emiliani1####################
Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 7

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 5

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 2

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 6

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 9

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 7

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 2

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 18

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 17

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 10

Number of feature

In [ ]:
basins = ['Dora', 'Piemonte_Nord', 'Piemonte_Sud']
path_target = "./csv/"
path_features_snow = './features_allvalues/snow/copernicus/relevant_coords/'
path_features='./features_allvalues/'

destination_folder = './NonLinCFA/temp_prec_snow_copernicus/relevant_coords/'

for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                                     path=path_target+basin+'.csv')

    eps = 0.01
    actual_path = path_features+basin+'_aggreg.csv'
    snow_actual_path = path_features_snow+basin+'_aggreg_sd_allCoord.csv'


    output, aggregate_trainVal, aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
    if os.path.isfile(snow_actual_path):
      output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS',
                                                                              'cyclostationary_mean_HS_1w',
                                                                              'cyclostationary_mean_HS_4w',
                                                                              'cyclostationary_mean_HS_8w',
                                                                              'cyclostationary_mean_HS_12w',
                                                                              'cyclostationary_mean_HS_16w',
                                                                              'cyclostationary_mean_HS_24w'
                                                                              ],
                                                                        target_df_trainVal, eps=eps,
                                                                        max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                                    aggreg_small_coord_groups = True, scale = 0.26)


      aggregate_trainVal = pd.concat((aggregate_trainVal_snow, aggregate_trainVal),axis=1)
      aggregate_test = pd.concat((aggregate_test_snow, aggregate_test),axis=1)

    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_train_withSnow.csv'
    val_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_val_withSnow.csv'
    test_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_test_withSnow.csv'

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_best5_CMI_train_withSnow.csv'
    val_string = destination_folder + basin + '_nonLinCFA_best5_CMI_val_withSnow.csv'
    test_string = destination_folder + basin + '_nonLinCFA_best5_CMI_test_withSnow.csv'

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

####################Dora####################
Number of features: 44

eps value:  0.00022727272727272727
Number of aggregated features: 3

Number of features: 44

eps value:  0.00022727272727272727
Number of aggregated features: 2

Number of features: 44

eps value:  0.00022727272727272727
Number of aggregated features: 2

Number of features: 44

eps value:  0.00022727272727272727
Number of aggregated features: 1

Number of features: 44

eps value:  0.00022727272727272727
Number of aggregated features: 3

Number of features: 44

eps value:  0.00022727272727272727
Number of aggregated features: 2

Number of features: 44

eps value:  0.00022727272727272727
Number of aggregated features: 1

Number of features: 44

eps value:  0.00022727272727272727
Number of aggregated features: 1

Number of features: 44

eps value:  0.00022727272727272727
Number of aggregated features: 1

Number of features: 44

eps value:  0.00022727272727272727
Number of aggregated features: 2

Number of features: 44

e

## Temp + Prec + Snow + Lakes

In [ ]:
basins = ['Adda', 'Ticino', 'Oglio_Iseo', 'Lambro_Olona']

path_target = "./csv/"
path_features_snow = './features_allvalues/snow/copernicus/'
path_features='./features_allvalues/'
path_features_lakes='./lakes/'

destination_folder = './NonLinCFA/temp_prec_snow_lakes_copernicus/'


for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')

    eps = 0.01
    actual_path = path_features+basin+'_aggreg.csv'
    snow_actual_path = path_features_snow+basin+'_aggreg_sd_allCoord.csv'
    lakes_actual_path = path_features_lakes + 'lakes_' + basin + '_with_aggreg.csv'

    output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')

    if os.path.isfile(snow_actual_path):
      output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS',
                                                                              'cyclostationary_mean_HS_1w',
                                                                              'cyclostationary_mean_HS_4w',
                                                                              'cyclostationary_mean_HS_8w',
                                                                              'cyclostationary_mean_HS_12w',
                                                                              'cyclostationary_mean_HS_16w',
                                                                              'cyclostationary_mean_HS_24w'
                                                                              ],
                                                                        target_df_trainVal, eps=eps,
                                                                        max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                                  aggreg_small_coord_groups = True, scale = 0.26)
      aggregate_trainVal = pd.concat((aggregate_trainVal_snow,aggregate_trainVal),axis=1)
      aggregate_test = pd.concat((aggregate_test_snow,aggregate_test),axis=1)

    if os.path.isfile(lakes_actual_path):
      df_lakes = pd.read_csv(lakes_actual_path)
      df_lakes = df_lakes.drop("Unnamed: 0", axis='columns')

      df_lakes_trainVal = df_lakes.loc[(df_lakes['date'] > '2001-01-01') & (df_lakes['date'] <= '2015-01-01'),:]
      df_lakes_trainVal.reset_index(inplace = True, drop = True)
      df_lakes_trainVal = df_lakes_trainVal.iloc[:,3:]
      df_lakes_test = df_lakes.loc[(df_lakes['date'] > '2015-01-01') & (df_lakes['date'] <= '2020-01-01'),:]
      df_lakes_test.reset_index(inplace = True, drop = True)
      df_lakes_test = df_lakes_test.iloc[:,3:]

      aggregate_trainVal = pd.concat((aggregate_trainVal, df_lakes_trainVal),axis=1)
      aggregate_test = pd.concat((aggregate_test, df_lakes_test),axis=1)

    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_train_withSnowLakes.csv'
    val_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_val_withSnowLakes.csv'
    test_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_test_withSnowLakes.csv'

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_best5_CMI_train_withSnowLakes.csv'
    val_string = destination_folder + basin + '_nonLinCFA_best5_CMI_val_withSnowLakes.csv'
    test_string = destination_folder + basin + '_nonLinCFA_best5_CMI_test_withSnowLakes.csv'

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    train_string = destination_folder + basin + '_nonLinCFA_CMI_train_withSnowLakes.csv'
    val_string = destination_folder + basin + '_nonLinCFA_CMI_val_withSnowLakes.csv'
    test_string = destination_folder + basin + '_nonLinCFA_CMI_test_withSnowLakes.csv'

    aggregate_trainVal.loc[:410,selected_colnames].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames].to_csv(test_string, index=False)

In [ ]:
basins = ['Adda', 'Ticino', 'Oglio_Iseo', 'Lambro_Olona']

path_target = "./csv/"
path_features_snow = './features_allvalues/snow/copernicus/relevant_coords/'
path_features='./features_allvalues/'

destination_folder = './NonLinCFA/temp_prec_snow_lakes_copernicus/relevant_coords/'
path_features_lakes='./lakes/'


for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')

    eps = 0.01
    actual_path = path_features+basin+'_aggreg.csv'
    snow_actual_path = path_features_snow+basin+'_aggreg_sd_allCoord.csv'
    lakes_actual_path = path_features_lakes + 'lakes_' + basin + '_with_aggreg.csv'

    output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')

    if os.path.isfile(snow_actual_path):
      output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS',
                                                                              'cyclostationary_mean_HS_1w',
                                                                              'cyclostationary_mean_HS_4w',
                                                                              'cyclostationary_mean_HS_8w',
                                                                              'cyclostationary_mean_HS_12w',
                                                                              'cyclostationary_mean_HS_16w',
                                                                              'cyclostationary_mean_HS_24w'
                                                                              ],
                                                                        target_df_trainVal, eps=eps,
                                                                        max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                                  aggreg_small_coord_groups = True, scale = 0.26)
      aggregate_trainVal = pd.concat((aggregate_trainVal_snow,aggregate_trainVal),axis=1)
      aggregate_test = pd.concat((aggregate_test_snow,aggregate_test),axis=1)

    if os.path.isfile(lakes_actual_path):
      df_lakes = pd.read_csv(lakes_actual_path)
      df_lakes = df_lakes.drop("Unnamed: 0", axis='columns')

      df_lakes_trainVal = df_lakes.loc[(df_lakes['date'] > '2001-01-01') & (df_lakes['date'] <= '2015-01-01'),:]
      df_lakes_trainVal.reset_index(inplace = True, drop = True)
      df_lakes_trainVal = df_lakes_trainVal.iloc[:,3:]
      df_lakes_test = df_lakes.loc[(df_lakes['date'] > '2015-01-01') & (df_lakes['date'] <= '2020-01-01'),:]
      df_lakes_test.reset_index(inplace = True, drop = True)
      df_lakes_test = df_lakes_test.iloc[:,3:]

      aggregate_trainVal = pd.concat((aggregate_trainVal, df_lakes_trainVal),axis=1)
      aggregate_test = pd.concat((aggregate_test, df_lakes_test),axis=1)

    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_train_withSnowLakes.csv'
    val_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_val_withSnowLakes.csv'
    test_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_test_withSnowLakes.csv'

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_best5_CMI_train_withSnowLakes.csv'
    val_string = destination_folder + basin + '_nonLinCFA_best5_CMI_val_withSnowLakes.csv'
    test_string = destination_folder + basin + '_nonLinCFA_best5_CMI_test_withSnowLakes.csv'

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    train_string = destination_folder + basin + '_nonLinCFA_CMI_train_withSnowLakes.csv'
    val_string = destination_folder + basin + '_nonLinCFA_CMI_val_withSnowLakes.csv'
    test_string = destination_folder + basin + '_nonLinCFA_CMI_test_withSnowLakes.csv'

    aggregate_trainVal.loc[:410,selected_colnames].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames].to_csv(test_string, index=False)

####################Adda####################
Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 4

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 5

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 4

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 1

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 3

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 1

Number of features: 92

e

## Temp + Prec + Lakes

In [ ]:
basins = ['Adda', 'Ticino', 'Oglio_Iseo']
path_target = "./csv/"
path_features='./features_allvalues/'
path_features_lakes='./lakes/'

destination_folder = './NonLinCFA/temp_prec_lakes/'


for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')

    eps = 0.001
    actual_path = path_features+basin+'_aggreg.csv'
    lakes_actual_path = path_features_lakes + 'lakes_' + basin + '_with_aggreg.csv'

    output,aggregate_trainVal_temp_prec,aggregate_test_temp_prec = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')

    if os.path.isfile(lakes_actual_path):
      df_lakes = pd.read_csv(lakes_actual_path)

      df_lakes = df_lakes.drop("Unnamed: 0", axis='columns')

      df_lakes_trainVal = df_lakes.loc[(df_lakes['date'] > '2001-01-01') & (df_lakes['date'] <= '2015-01-01'),:]
      df_lakes_trainVal.reset_index(inplace = True, drop = True)
      df_lakes_trainVal = df_lakes_trainVal.iloc[:,3:]
      df_lakes_test = df_lakes.loc[(df_lakes['date'] > '2015-01-01') & (df_lakes['date'] <= '2020-01-01'),:]
      df_lakes_test.reset_index(inplace = True, drop = True)
      df_lakes_test = df_lakes_test.iloc[:,3:]

      aggregate_trainVal = pd.concat((df_lakes_trainVal,aggregate_trainVal_temp_prec),axis=1)
      aggregate_test = pd.concat((df_lakes_test,aggregate_test_temp_prec),axis=1)

    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_train_withLakes.csv'
    val_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_val_withLakes.csv'
    test_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_test_withLakes.csv'

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_best5_CMI_train_withLakes.csv'
    val_string = destination_folder + basin + '_nonLinCFA_best5_CMI_val_withLakes.csv'
    test_string = destination_folder + basin + '_nonLinCFA_best5_CMI_test_withLakes.csv'

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    train_string = destination_folder + basin + '_nonLinCFA_CMI_train_withLakes.csv'
    val_string = destination_folder + basin + '_nonLinCFA_CMI_val_withLakes.csv'
    test_string = destination_folder + basin + '_nonLinCFA_CMI_test_withLakes.csv'

    aggregate_trainVal.loc[:410,selected_colnames].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames].to_csv(test_string, index=False)

####################Adda####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1 -2.546951
1    2001-01-13  0.380618    0.43  2001     2 -0.277191
2    2001-01-21  0.341985    0.38  2001     3 -0.534156
3    2001-01-29  0.322044    0.35  2001     5 -0.666789
4    2001-02-06  0.354954    0.40  2001     6 -0.447894
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.382706    0.40  2009    48 -0.263306
407  2009-12-05  0.409921    0.46  2009    49 -0.082282
408  2009-12-13  0.472087    0.53  2009    50  0.331204
409  2009-12-21  0.324728    0.00  2009    52 -0.648940
410  2009-12-29  0.086512    0.00  2009    53 -2.233412

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 92

Number of aggregated features: 8

Number of features: 92

Number of aggregated features: 9

Number of features: 92

Number of aggregated features: 5

Number of fe

# NonLinCFA results in binary classification

## Target and features

In [ ]:
cd ..

/content/drive/MyDrive/Colab Notebooks


In [ ]:
### targets
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
path_targets = "./csv/"
targets_df_train = pd.DataFrame()
targets_df_val = pd.DataFrame()
targets_df_test = pd.DataFrame()
targets_df_trainVal = pd.DataFrame()

for basin in basins:
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                                     path=path_targets+basin+'.csv', window_size = 4)
    targets_df_train[basin] = target_df_train.apply(lambda x: np.sign(x.mean_std), axis=1)
    targets_df_val[basin] = target_df_val.apply(lambda x: np.sign(x.mean_std), axis=1)
    targets_df_test[basin] = target_df_test.apply(lambda x: np.sign(x.mean_std), axis=1)
    targets_df_trainVal[basin] = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)

target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1 -3.170273
1    2001-01-13  0.209996    0.43  2001     2 -1.756328
2    2001-01-21  0.253992    0.38  2001     3 -1.391730
3    2001-01-29  0.271005    0.35  2001     5 -1.250743
4    2001-02-06  0.349900    0.40  2001     6 -0.596941
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.449148    0.40  2009    48  0.225525
407  2009-12-05  0.405264    0.46  2009    49 -0.138143
408  2009-12-13  0.397738    0.53  2009    50 -0.200511
409  2009-12-21  0.397360    0.00  2009    52 -0.203640
410  2009-12-29  0.323312    0.00  2009    53 -0.817278

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.010645    0.00  2001     1 -2.401158
1    2001-01-13  0.108707    0.00  2001     2 -1.723004
2    2001-01-21  0.161576    0.00  2001     3 -

In [ ]:
def get_features (path_features, basin, train_name, val_name, test_name):
  features_train = pd.read_csv(path_features + basin + train_name)
  features_valid = pd.read_csv(path_features + basin + val_name)
  features_test = pd.read_csv(path_features + basin + test_name)

  features_train_validation = pd.concat([features_train, features_valid])
  return features_train.values, features_valid.values, features_train_validation.values, features_test.values

In [ ]:
basin = 'Garda_Mincio'

In [ ]:
# Temp Prec
path_features = './NonLinCFA/temp_prec_moving_avg_2/'

# CMI
X_train_CMI, X_validation_CMI, X_train_validation_CMI, X_test_CMI = get_features(path_features, basin, '_nonLinCFA_CMI_train.csv',
                                                                                 '_nonLinCFA_CMI_val.csv', '_nonLinCFA_CMI_test.csv')

# CMI_best_5
X_train_CMI5, X_validation_CMI5, X_train_validation_CMI5, X_test_CMI5 = get_features(path_features, basin, '_nonLinCFA_best5_CMI_train.csv',
                                                                                     '_nonLinCFA_best5_CMI_val.csv', '_nonLinCFA_best5_CMI_test.csv')

# wrapper
X_train_wrapper, X_validation_wrapper, X_train_validation_wrapper, X_test_wrapper = get_features(path_features, basin, '_nonLinCFA_wrapper_best5_train.csv',
                                                                                                 '_nonLinCFA_wrapper_best5_val.csv', '_nonLinCFA_wrapper_best5_test.csv')

In [ ]:
print ("Features chosen by CMI for " + basin + " taken temp, prec:")
path_features = './NonLinCFA/temp_prec/'
features_df = pd.read_csv(path_features + basin + '_nonLinCFA_best5_CMI_train.csv')
idx_dic = {}
for col in features_df.columns:
    idx_dic[col] = features_df.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

Features chosen by CMI for Garda_Mincio taken temp, prec:
0: cyclostationary_mean_rr_4w_0
1: cyclostationary_mean_tg_0
2: cyclostationary_mean_rr_8w_0
3: cyclostationary_mean_rr_12w_2
4: cyclostationary_mean_tg_8w_0


In [ ]:
print ("Features chosen by CMI for " + basin + " taken temp, prec:")
path_features = './NonLinCFA/temp_prec_moving_avg_2/'
features_df = pd.read_csv(path_features + basin + '_nonLinCFA_best5_CMI_train.csv')
idx_dic = {}
for col in features_df.columns:
    idx_dic[col] = features_df.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

Features chosen by CMI for Garda_Mincio taken temp, prec:
0: cyclostationary_mean_rr_8w_0
1: cyclostationary_mean_tg_1w_0
2: cyclostationary_mean_rr_4w_2
3: cyclostationary_mean_rr_16w_4
4: cyclostationary_mean_tg_12w_0


In [ ]:
# Temp Prec Snow
path_features = './NonLinCFA/temp_prec_snow/'

# CMI
X_train_CMI, X_validation_CMI, X_train_validation_CMI, X_test_CMI = get_features(path_features, basin, '_nonLinCFA_CMI_train_withSnow.csv',
                                                                                 '_nonLinCFA_CMI_val_withSnow.csv', '_nonLinCFA_CMI_test_withSnow.csv')

# CMI_best_5
X_train_CMI5, X_validation_CMI5, X_train_validation_CMI5, X_test_CMI5 = get_features(path_features, basin, '_nonLinCFA_best5_CMI_train_withSnow.csv',
                                                                                     '_nonLinCFA_best5_CMI_val_withSnow.csv', '_nonLinCFA_best5_CMI_test_withSnow.csv')

# wrapper
X_train_wrapper, X_validation_wrapper, X_train_validation_wrapper, X_test_wrapper = get_features(path_features, basin, '_wrapper_best5_train_withSnow.csv',
                                                                                                 '_wrapper_best5_val_withSnow.csv', '_wrapper_best5_test_withSnow.csv')

In [ ]:
print ("Features chosen by CMI for " + basin + " taken temp, prec, snow:")
path_features = './NonLinCFA/temp_prec_snow/'
features_df = pd.read_csv(path_features + basin + '_nonLinCFA_best5_CMI_val_withSnow.csv')
idx_dic = {}
for col in features_df.columns:
    idx_dic[col] = features_df.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

Features chosen by CMI for Ticino taken temp, prec, snow:
0: cyclostationary_mean_tg_0
1: cyclostationary_mean_HS_0
2: cyclostationary_mean_rr_1w_0
3: cyclostationary_mean_HS_1w_0


In [ ]:
print ("Features chosen by wrapper for " + basin + " taken temp, prec, snow:")
path_features = './NonLinCFA/temp_prec_snow/'
features_df = pd.read_csv(path_features + basin + '_wrapper_best5_val_withSnow.csv')
idx_dic = {}
for col in features_df.columns:
    idx_dic[col] = features_df.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

Features chosen by wrapper for Ticino taken temp, prec, snow:
0: cyclostationary_mean_tg_1w_3
1: cyclostationary_mean_rr_1w_1
2: cyclostationary_mean_HS_12w_1
3: cyclostationary_mean_rr_24w_0
4: cyclostationary_mean_tg_16w_3


In [ ]:
# Temp Prec Snow Lakes
path_features = './NonLinCFA/temp_prec_snow_lakes/'

# CMI
X_train_CMI, X_validation_CMI, X_train_validation_CMI, X_test_CMI = get_features(path_features, basin, '_nonLinCFA_CMI_train_withSnowLakes.csv',
                                                                                 '_nonLinCFA_CMI_val_withSnowLakes.csv', '_nonLinCFA_CMI_test_withSnowLakes.csv')

# CMI_best_5
X_train_CMI5, X_validation_CMI5, X_train_validation_CMI5, X_test_CMI5 = get_features(path_features, basin, '_nonLinCFA_best5_CMI_train_withSnowLakes.csv',
                                                                                     '_nonLinCFA_best5_CMI_val_withSnowLakes.csv', '_nonLinCFA_best5_CMI_test_withSnowLakes.csv')

# wrapper
X_train_wrapper, X_validation_wrapper, X_train_validation_wrapper, X_test_wrapper = get_features(path_features, basin, '_wrapper_best5_train_withSnowLakes.csv',
                                                                                                 '_wrapper_best5_val_withSnowLakes.csv', '_wrapper_best5_test_withSnowLakes.csv')

In [ ]:
print ("Features chosen by wrapper for " + basin + " \n taken temp, prec, lakes, snow:")
path_features = './NonLinCFA/temp_prec_snow_lakes/'
features_df = pd.read_csv(path_features + basin + '_wrapper_best5_train_withSnowLakes.csv')
idx_dic = {}
for col in features_df.columns:
    idx_dic[col] = features_df.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

Features chosen by wrapper for Ticino 
 taken temp, prec, lakes, snow:
0: cyclostationary_mean_tg_1w_3
1: cyclostationary_mean_rr_1w_1
2: cyclostationary_mean_HS_12w_1
3: cyclostat_release_Lugano_24w
4: cyclostationary_mean_tg_16w_3


In [ ]:
# Temp Prec Lakes
path_features = './NonLinCFA/temp_prec_lakes/'

# CMI
X_train_CMI, X_validation_CMI, X_train_validation_CMI, X_test_CMI = get_features(path_features, basin, '_nonLinCFA_CMI_train_withLakes.csv',
                                                                                 '_nonLinCFA_CMI_val_withLakes.csv', '_nonLinCFA_CMI_test_withLakes.csv')

# CMI_best_5
X_train_CMI5, X_validation_CMI5, X_train_validation_CMI5, X_test_CMI5 = get_features(path_features, basin, '_nonLinCFA_best5_CMI_train_withLakes.csv',
                                                                                     '_nonLinCFA_best5_CMI_val_withLakes.csv', '_nonLinCFA_best5_CMI_test_withLakes.csv')

# wrapper
X_train_wrapper, X_validation_wrapper, X_train_validation_wrapper, X_test_wrapper = get_features(path_features, basin, '_wrapper_best5_train_withLakes.csv',
                                                                                                 '_wrapper_best5_val_withLakes.csv', '_wrapper_best5_test_withLakes.csv')

In [ ]:
print ("Features chosen by wrapper for " + basin + " \ntaken temp, prec, lakes:")
features_df = pd.read_csv(path_features + basin + '_wrapper_best5_train_withLakes.csv')
idx_dic = {}
for col in features_df.columns:
    idx_dic[col] = features_df.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

Features chosen by wrapper for Oglio_Iseo 
taken temp, prec, lakes:
0: cyclostationary_mean_tg_1w_4
1: cyclostat_level_Iseo_4w
2: cyclostationary_mean_rr_16w_0
3: cyclostationary_mean_rr_1w_0
4: cyclostat_level_Iseo


## Single Task

In [ ]:
log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

# CMI
log_regr.fit(X_train_validation_CMI, targets_df_trainVal[basin])
print("Train accuracy logistic regression CMI: ", round(log_regr.score(X_train_validation_CMI, targets_df_trainVal[basin]),3))
print("Test accuracy logistic regression CMI: ", round(log_regr.score(X_test_CMI, targets_df_test[basin]),3), "\n")

# CMI best 5
log_regr.fit(X_train_validation_CMI5, targets_df_trainVal[basin])
print("Train accuracy logistic regression CMI best 5: ", round(log_regr.score(X_train_validation_CMI5, targets_df_trainVal[basin]),3))
print("Test accuracy logistic regression CMI best 5: ", round(log_regr.score(X_test_CMI5, targets_df_test[basin]),3), "\n")

# wrapper
log_regr.fit(X_train_validation_wrapper, targets_df_trainVal[basin])
print("Train accuracy logistic regression wrapper: ", round(log_regr.score(X_train_validation_wrapper, targets_df_trainVal[basin]),3))
print("Test accuracy logistic regression wrapper: ", round(log_regr.score(X_test_wrapper, targets_df_test[basin]),3), "\n")

Train accuracy logistic regression CMI:  0.775
Test accuracy logistic regression CMI:  0.728 

Train accuracy logistic regression CMI best 5:  0.762
Test accuracy logistic regression CMI best 5:  0.768 

Train accuracy logistic regression wrapper:  0.768
Test accuracy logistic regression wrapper:  0.706 



## Multi Task

In [ ]:
def MTL_scores(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test):

    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)

    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()

    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_ohe = LogisticRegression(max_iter = 500)
    model_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)).values,pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)).values.ravel())

    for basin in clust_basins:
        print(basin)
        res = model_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
        print(accuracy_score(targets_df_test[basin].values.ravel(), res))

### Features

#### Temp Prec

In [ ]:
### wrapper best 5 features
path_features = './NonLinCFA/temp_prec_moving_avg_4/'

best5_wrapper_fulldf_train = pd.DataFrame()
best5_wrapper_fulldf_val = pd.DataFrame()
best5_wrapper_fulldf_test = pd.DataFrame()

for basin in basins:
    best5_wrapper_train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_train.csv')
    best5_wrapper_val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_val.csv')
    best5_wrapper_test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_test.csv')
    best5_wrapper_fulldf_train[basin+'_'+best5_wrapper_train_temp.columns.values] = best5_wrapper_train_temp
    best5_wrapper_fulldf_val[basin+'_'+best5_wrapper_val_temp.columns.values] = best5_wrapper_val_temp
    best5_wrapper_fulldf_test[basin+'_'+best5_wrapper_test_temp.columns.values] = best5_wrapper_test_temp

In [ ]:
### CMI features
path_features = './NonLinCFA/temp_prec_moving_avg_4/'

CMI_fulldf_train = pd.DataFrame()
CMI_fulldf_val = pd.DataFrame()
CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_train.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_val.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_test.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
### CMI best5 features
path_features = './NonLinCFA/temp_prec_moving_avg_4/'

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

#### Temp Prec Snow

In [ ]:
### wrapper best 5 features
path_features = './NonLinCFA/temp_prec_snow/'

#best5_wrapper_fulldf_train = pd.DataFrame()
#best5_wrapper_fulldf_val = pd.DataFrame()
#best5_wrapper_fulldf_test = pd.DataFrame()
basins_snow = ['Piemonte_Nord', 'Piemonte_Sud', 'Ticino', 'Dora', 'Adda']

for basin in basins_snow:
    best5_wrapper_train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_train_withSnow.csv')
    best5_wrapper_val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_val_withSnow.csv')
    best5_wrapper_test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_test_withSnow.csv')
    best5_wrapper_fulldf_train[basin+'_'+best5_wrapper_train_temp.columns.values] = best5_wrapper_train_temp
    best5_wrapper_fulldf_val[basin+'_'+best5_wrapper_val_temp.columns.values] = best5_wrapper_val_temp
    best5_wrapper_fulldf_test[basin+'_'+best5_wrapper_test_temp.columns.values] = best5_wrapper_test_temp

In [ ]:
### CMI features
path_features = './NonLinCFA/temp_prec_snow/'

#CMI_fulldf_train = pd.DataFrame() commentate per mantenere i dati dei bacini senza neve
#CMI_fulldf_val = pd.DataFrame()
#CMI_fulldf_test = pd.DataFrame()

for basin in basins_snow:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_train_withSnow.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_val_withSnow.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_test_withSnow.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
### CMI best5 features
path_features = './NonLinCFA/temp_prec_snow/'

#best5_CMI_fulldf_train = pd.DataFrame()
#best5_CMI_fulldf_val = pd.DataFrame()
#best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins_snow:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train_withSnow.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val_withSnow.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test_withSnow.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

#### Temp Prec Snow Lakes

In [ ]:
### wrapper best 5 features
path_features = './NonLinCFA/temp_prec_snow_lakes/'

#best5_wrapper_fulldf_train = pd.DataFrame()
#best5_wrapper_fulldf_val = pd.DataFrame()
#best5_wrapper_fulldf_test = pd.DataFrame()
basins_lakes = ['Adda', 'Oglio_Iseo', 'Ticino']

for basin in basins_lakes:
    best5_wrapper_train_temp = pd.read_csv(path_features+basin+'_wrapper_best5_train_withSnowLakes.csv')
    best5_wrapper_val_temp = pd.read_csv(path_features+basin+'_wrapper_best5_val_withSnowLakes.csv')
    best5_wrapper_test_temp = pd.read_csv(path_features+basin+'_wrapper_best5_test_withSnowLakes.csv')
    best5_wrapper_fulldf_train[basin+'_'+best5_wrapper_train_temp.columns.values] = best5_wrapper_train_temp
    best5_wrapper_fulldf_val[basin+'_'+best5_wrapper_val_temp.columns.values] = best5_wrapper_val_temp
    best5_wrapper_fulldf_test[basin+'_'+best5_wrapper_test_temp.columns.values] = best5_wrapper_test_temp

In [ ]:
### CMI features
path_features = './NonLinCFA/temp_prec_snow_lakes/'

#CMI_fulldf_train = pd.DataFrame()
#CMI_fulldf_val = pd.DataFrame()
#CMI_fulldf_test = pd.DataFrame()

for basin in basins_lakes:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_train_withSnowLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_val_withSnowLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_test_withSnowLakes.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
### CMI best5 features
path_features = './NonLinCFA/temp_prec_snow_lakes/'

#best5_CMI_fulldf_train = pd.DataFrame()
#best5_CMI_fulldf_val = pd.DataFrame()
#best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins_lakes:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train_withSnowLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val_withSnowLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test_withSnowLakes.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

#### Temp Prec Lakes

In [ ]:
### wrapper best 5 features
path_features = './NonLinCFA/temp_prec_lakes/'

#best5_wrapper_fulldf_train = pd.DataFrame()
#best5_wrapper_fulldf_val = pd.DataFrame()
#best5_wrapper_fulldf_test = pd.DataFrame()

basins_lakes = ['Adda', 'Oglio_Iseo', 'Ticino']

for basin in basins_lakes:
    best5_wrapper_train_temp = pd.read_csv(path_features+basin+'_wrapper_best5_train_withLakes.csv')
    best5_wrapper_val_temp = pd.read_csv(path_features+basin+'_wrapper_best5_val_withLakes.csv')
    best5_wrapper_test_temp = pd.read_csv(path_features+basin+'_wrapper_best5_test_withLakes.csv')
    best5_wrapper_fulldf_train[basin+'_'+best5_wrapper_train_temp.columns.values] = best5_wrapper_train_temp
    best5_wrapper_fulldf_val[basin+'_'+best5_wrapper_val_temp.columns.values] = best5_wrapper_val_temp
    best5_wrapper_fulldf_test[basin+'_'+best5_wrapper_test_temp.columns.values] = best5_wrapper_test_temp

In [ ]:
### CMI features
path_features = './NonLinCFA/temp_prec_lakes/'

#CMI_fulldf_train = pd.DataFrame()
#CMI_fulldf_val = pd.DataFrame()
#CMI_fulldf_test = pd.DataFrame()

for basin in basins_lakes:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_train_withLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_val_withLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_test_withLakes.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
### CMI best5 features
path_features = './NonLinCFA/temp_prec_lakes/'

#best5_CMI_fulldf_train = pd.DataFrame()
#best5_CMI_fulldf_val = pd.DataFrame()
#best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins_lakes:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train_withLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val_withLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test_withLakes.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

### Wrapper

In [ ]:
MTL_scores(clust_basins=['Emiliani1', 'Emiliani2', 'Garda_Mincio'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
0.6447368421052632
Emiliani2
0.6842105263157895
Garda_Mincio
0.6798245614035088


In [ ]:
MTL_scores(clust_basins=['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Adda
0.7149122807017544
Lambro_Olona
0.6754385964912281
Oglio_Iseo
0.6578947368421053
Ticino
0.7017543859649122


In [ ]:
MTL_scores(clust_basins=['Dora', 'Piemonte_Nord', 'Piemonte_Sud'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
0.6271929824561403
Piemonte_Nord
0.706140350877193
Piemonte_Sud
0.7192982456140351


In [ ]:
MTL_scores(clust_basins=['Piemonte_Nord', 'Piemonte_Sud'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Piemonte_Nord
0.6973684210526315
Piemonte_Sud
0.7149122807017544


### CMI

In [ ]:
MTL_scores(clust_basins=['Emiliani1', 'Emiliani2', 'Garda_Mincio'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
0.6535087719298246
Emiliani2
0.6271929824561403
Garda_Mincio
0.75


In [ ]:
MTL_scores(clust_basins=['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Adda
0.6052631578947368
Lambro_Olona
0.5482456140350878
Oglio_Iseo
0.5789473684210527
Ticino
0.5789473684210527


In [ ]:
MTL_scores(clust_basins=['Dora', 'Piemonte_Nord', 'Piemonte_Sud'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
0.5219298245614035
Piemonte_Nord
0.631578947368421
Piemonte_Sud
0.7149122807017544


In [ ]:
MTL_scores(clust_basins=['Piemonte_Nord', 'Piemonte_Sud'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Piemonte_Nord
0.6754385964912281
Piemonte_Sud
0.7105263157894737


### CMI best 5

In [ ]:
MTL_scores(clust_basins=['Emiliani1', 'Emiliani2', 'Garda_Mincio'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
0.6447368421052632
Emiliani2
0.618421052631579
Garda_Mincio
0.7412280701754386


In [ ]:
MTL_scores(clust_basins=['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Adda
0.6754385964912281
Lambro_Olona
0.6359649122807017
Oglio_Iseo
0.631578947368421
Ticino
0.6578947368421053


In [ ]:
MTL_scores(clust_basins=['Dora', 'Piemonte_Nord', 'Piemonte_Sud'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
0.6140350877192983
Piemonte_Nord
0.7412280701754386
Piemonte_Sud
0.7236842105263158


In [ ]:
MTL_scores(clust_basins=['Piemonte_Nord', 'Piemonte_Sud'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Piemonte_Nord
0.6973684210526315
Piemonte_Sud
0.6622807017543859


# NonLinCFA results in linear regression

## Target and features

In [ ]:
### targets
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
path_targets = "./csv/"
targets_df_train = pd.DataFrame()
targets_df_val = pd.DataFrame()
targets_df_test = pd.DataFrame()
targets_df_trainVal = pd.DataFrame()

for basin in basins:
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01',
                                                                                     max_test='2020-01-01', path=path_targets+basin+'.csv', window_size = 1)
    targets_df_train[basin] = target_df_train.mean_std
    targets_df_val[basin] = target_df_val.mean_std
    targets_df_test[basin] = target_df_test.mean_std
    targets_df_trainVal[basin] = target_df_trainVal.mean_std

In [ ]:
def get_features (path_features, basin, train_name, val_name, test_name):
  features_train = pd.read_csv(path_features + basin + train_name)
  features_valid = pd.read_csv(path_features + basin + val_name)
  features_test = pd.read_csv(path_features + basin + test_name)

  features_train_validation = pd.concat([features_train, features_valid])
  return features_train.values, features_valid.values, features_train_validation.values, features_test.values

In [ ]:
basin = 'Piemonte_Sud'

In [ ]:
# Temp Prec
path_features = './NonLinCFA/temp_prec_internal_ordering/dyn_eps_lin/'

# CMI
#X_train_CMI, X_validation_CMI, X_train_validation_CMI, X_test_CMI = get_features(path_features, basin, '_nonLinCFA_CMI_train.csv',
#                                                                                 '_nonLinCFA_CMI_val.csv', '_nonLinCFA_CMI_test.csv')

# CMI_best_5
X_train_CMI5, X_validation_CMI5, X_train_validation_CMI5, X_test_CMI5 = get_features(path_features, basin, '_nonLinCFA_best5_CMI_train.csv',
                                                                                     '_nonLinCFA_best5_CMI_val.csv', '_nonLinCFA_best5_CMI_test.csv')

# wrapper
#X_train_wrapper, X_validation_wrapper, X_train_validation_wrapper, X_test_wrapper = get_features(path_features, basin, '_nonLinCFA_wrapper_best5_train.csv',
#                                                                                                 '_nonLinCFA_wrapper_best5_val.csv', '_nonLinCFA_wrapper_best5_test.csv')

In [ ]:
print ("Features chosen by CMI best 5 for " + basin + " \ntaken " + "\033[1m" + "temp, prec"+ "\033[0m" + ":")
path_features = './NonLinCFA/temp_prec_internal_ordering/dyn_eps_lin/'
features_df = pd.read_csv(path_features + basin + '_nonLinCFA_best5_CMI_train.csv')
idx_dic = {}
for col in features_df.columns:
    idx_dic[col] = features_df.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

Features chosen by CMI best 5 for Piemonte_Sud 
taken temp, prec:
0: cyclostationary_mean_tg_0
1: cyclostationary_mean_rr_24w_0
2: cyclostationary_mean_rr_8w_1
3: cyclostationary_mean_rr_16w_0
4: cyclostationary_mean_tg_24w_0


In [ ]:
# Temp Prec Snow
path_features = './NonLinCFA/temp_prec_snow_copernicus/'

# CMI
#X_train_CMI, X_validation_CMI, X_train_validation_CMI, X_test_CMI = get_features(path_features, basin, '_nonLinCFA_CMI_train_withSnow.csv',
#                                                                                 '_nonLinCFA_CMI_val_withSnow.csv', '_nonLinCFA_CMI_test_withSnow.csv')

# CMI_best_5
X_train_CMI5, X_validation_CMI5, X_train_validation_CMI5, X_test_CMI5 = get_features(path_features, basin, '_nonLinCFA_best5_CMI_train_withSnow.csv',
                                                                                     '_nonLinCFA_best5_CMI_val_withSnow.csv', '_nonLinCFA_best5_CMI_test_withSnow.csv')

# wrapper
#X_train_wrapper, X_validation_wrapper, X_train_validation_wrapper, X_test_wrapper = get_features(path_features, basin, '_wrapper_best5_train_withSnow.csv',
#                                                                                                 '_wrapper_best5_val_withSnow.csv', '_wrapper_best5_test_withSnow.csv')

In [ ]:
print ("Features chosen by CMI best 5 for " + basin + " \ntaken " + "\033[1m" + "temp, prec, snow"+ "\033[0m" + ":")
path_features = './NonLinCFA/temp_prec_snow_copernicus/'
features_df = pd.read_csv(path_features + basin + '_nonLinCFA_best5_CMI_train_withSnow.csv')
idx_dic = {}
for col in features_df.columns:
    idx_dic[col] = features_df.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

Features chosen by CMI best 5 for Piemonte_Sud 
taken temp, prec, snow:
0: cyclostationary_mean_HS_1
1: cyclostationary_mean_tg_1w_0
2: cyclostationary_mean_rr_12w_2
3: cyclostationary_mean_HS_0
4: cyclostationary_mean_tg_8w_3


In [ ]:
# Temp Prec Snow Lakes
path_features = './NonLinCFA/temp_prec_snow_lakes_copernicus/'

# CMI
#X_train_CMI, X_validation_CMI, X_train_validation_CMI, X_test_CMI = get_features(path_features, basin, '_nonLinCFA_CMI_train_withSnowLakes.csv',
#                                                                                 '_nonLinCFA_CMI_val_withSnowLakes.csv', '_nonLinCFA_CMI_test_withSnowLakes.csv')

# CMI_best_5
X_train_CMI5, X_validation_CMI5, X_train_validation_CMI5, X_test_CMI5 = get_features(path_features, basin, '_nonLinCFA_best5_CMI_train_withSnowLakes.csv',
                                                                                     '_nonLinCFA_best5_CMI_val_withSnowLakes.csv', '_nonLinCFA_best5_CMI_test_withSnowLakes.csv')

# wrapper
#X_train_wrapper, X_validation_wrapper, X_train_validation_wrapper, X_test_wrapper = get_features(path_features, basin, '_wrapper_best5_train_withSnowLakes.csv',
#                                                                                                 '_wrapper_best5_val_withSnowLakes.csv', '_wrapper_best5_test_withSnowLakes.csv')

In [ ]:
basin = 'Lambro_Olona'
print ("Features chosen by CMI best 5 for " + basin + " \ntaken " + "\033[1m" + "temp, prec, snow, lakes"+ "\033[0m" + ":")
path_features = './NonLinCFA/temp_prec_snow_lakes_copernicus/'
features_df = pd.read_csv(path_features + basin + '_nonLinCFA_best5_CMI_train_withSnowLakes.csv')
idx_dic = {}
for col in features_df.columns:
    idx_dic[col] = features_df.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

Features chosen by CMI best 5 for Lambro_Olona 
taken temp, prec, snow, lakes:
0: cyclostationary_mean_tg_1w_5
1: cyclostationary_mean_HS_1w_0
2: cyclostationary_mean_rr_1w_1
3: cyclostationary_mean_HS_0
4: cyclostationary_mean_HS_8w_0


In [ ]:
# Temp Prec Lakes
path_features = './NonLinCFA/temp_prec_lakes/'

# CMI
X_train_CMI, X_validation_CMI, X_train_validation_CMI, X_test_CMI = get_features(path_features, basin, '_nonLinCFA_CMI_train_withLakes.csv',
                                                                                 '_nonLinCFA_CMI_val_withLakes.csv', '_nonLinCFA_CMI_test_withLakes.csv')

# CMI_best_5
X_train_CMI5, X_validation_CMI5, X_train_validation_CMI5, X_test_CMI5 = get_features(path_features, basin, '_nonLinCFA_best5_CMI_train_withLakes.csv',
                                                                                     '_nonLinCFA_best5_CMI_val_withLakes.csv', '_nonLinCFA_best5_CMI_test_withLakes.csv')

# wrapper
X_train_wrapper, X_validation_wrapper, X_train_validation_wrapper, X_test_wrapper = get_features(path_features, basin, '_wrapper_best5_train_withLakes.csv',
                                                                                                 '_wrapper_best5_val_withLakes.csv', '_wrapper_best5_test_withLakes.csv')

## Single Task

In [ ]:
lin_regr = LinearRegression()

# CMI
lin_regr.fit(X_train_validation_CMI, targets_df_trainVal[basin])
print("Train R2 linear regression CMI: ", round(lin_regr.score(X_train_validation_CMI, targets_df_trainVal[basin]),3))
print("Test R2 linear regression CMI: ", round(lin_regr.score(X_test_CMI, targets_df_test[basin]),3), "\n")

# CMI best 5
lin_regr.fit(X_train_validation_CMI5, targets_df_trainVal[basin])
print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(X_train_validation_CMI5, targets_df_trainVal[basin]),3))
print("Test R2 linear regression CMI best 5: ", round(lin_regr.score(X_test_CMI5, targets_df_test[basin]),3), "\n")

# wrapper
lin_regr.fit(X_train_validation_wrapper, targets_df_trainVal[basin])
print("Train R2 linear regression wrapper: ", round(lin_regr.score(X_train_validation_wrapper, targets_df_trainVal[basin]),3))
print("Test R2 linear regression wrapper: ", round(lin_regr.score(X_test_wrapper, targets_df_test[basin]),3), "\n")

Train R2 linear regression CMI:  0.237
Test R2 linear regression CMI:  0.159 

Train R2 linear regression CMI best 5:  0.219
Test R2 linear regression CMI best 5:  0.197 

Train R2 linear regression wrapper:  0.233
Test R2 linear regression wrapper:  0.146 



In [ ]:
basin

'Garda_Mincio'

In [ ]:
lin_regr = LinearRegression()
# CMI best 5
lin_regr.fit(X_train_validation_CMI5, targets_df_trainVal[basin])
print("Train accuracy logistic regression CMI best 5: ", round(lin_regr.score(X_train_validation_CMI5, targets_df_trainVal[basin]),3))
print("Test accuracy logistic regression CMI best 5: ", round(lin_regr.score(X_test_CMI5, targets_df_test[basin]),3), "\n")

Train accuracy logistic regression CMI best 5:  0.177
Test accuracy logistic regression CMI best 5:  0.216 



In [ ]:
# wrapper
lin_regr.fit(X_train_wrapper, targets_df_train[basin])
print("Train accuracy logistic regression wrapper: ", round(lin_regr.score(X_train_wrapper, targets_df_train[basin]),3))
print("Test accuracy logistic regression wrapper: ", round(lin_regr.score(X_validation_wrapper, targets_df_val[basin]),3), "\n")

Train accuracy logistic regression wrapper:  0.157
Test accuracy logistic regression wrapper:  0.344 



## Multi Task

In [ ]:
def MTL_scores(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test):

    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)

    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()

    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_ohe = LinearRegression()
    model_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)).values,pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)).values.ravel())

    for basin in clust_basins:
        print(basin)
        res = model_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
        print(r2_score(targets_df_test[basin].values.ravel(), res))

### Features

#### Temp Prec

In [ ]:
### wrapper best 5 features
path_features = './NonLinCFA/temp_prec_moving_avg_4/'

best5_wrapper_fulldf_train = pd.DataFrame()
best5_wrapper_fulldf_val = pd.DataFrame()
best5_wrapper_fulldf_test = pd.DataFrame()

for basin in basins:
    best5_wrapper_train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_train.csv')
    best5_wrapper_val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_val.csv')
    best5_wrapper_test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_test.csv')
    best5_wrapper_fulldf_train[basin+'_'+best5_wrapper_train_temp.columns.values] = best5_wrapper_train_temp
    best5_wrapper_fulldf_val[basin+'_'+best5_wrapper_val_temp.columns.values] = best5_wrapper_val_temp
    best5_wrapper_fulldf_test[basin+'_'+best5_wrapper_test_temp.columns.values] = best5_wrapper_test_temp

In [ ]:
### CMI features
path_features = './NonLinCFA/temp_prec/'

CMI_fulldf_train = pd.DataFrame()
CMI_fulldf_val = pd.DataFrame()
CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_train.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_val.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_test.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
### CMI best5 features
basins = ['Emiliani1','Emiliani2','Garda_Mincio']
path_features = './NonLinCFA/temp_prec_inverted_for/bottom_right/'

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

#### Temp Prec Snow

In [ ]:
### wrapper best 5 features
path_features = './NonLinCFA/temp_prec_snow/'

#best5_wrapper_fulldf_train = pd.DataFrame()
#best5_wrapper_fulldf_val = pd.DataFrame()
#best5_wrapper_fulldf_test = pd.DataFrame()
basins_snow = ['Piemonte_Nord', 'Piemonte_Sud', 'Dora']

for basin in basins_snow:
    best5_wrapper_train_temp = pd.read_csv(path_features+basin+'_wrapper_best5_train_withSnow.csv')
    best5_wrapper_val_temp = pd.read_csv(path_features+basin+'_wrapper_best5_val_withSnow.csv')
    best5_wrapper_test_temp = pd.read_csv(path_features+basin+'_wrapper_best5_test_withSnow.csv')
    best5_wrapper_fulldf_train[basin+'_'+best5_wrapper_train_temp.columns.values] = best5_wrapper_train_temp
    best5_wrapper_fulldf_val[basin+'_'+best5_wrapper_val_temp.columns.values] = best5_wrapper_val_temp
    best5_wrapper_fulldf_test[basin+'_'+best5_wrapper_test_temp.columns.values] = best5_wrapper_test_temp

In [ ]:
### CMI features
path_features = './NonLinCFA/temp_prec_snow/'

#CMI_fulldf_train = pd.DataFrame() commentate per mantenere i dati dei bacini senza neve
#CMI_fulldf_val = pd.DataFrame()
#CMI_fulldf_test = pd.DataFrame()

for basin in basins_snow:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_train_withSnow.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_val_withSnow.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_test_withSnow.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
### CMI best5 features
path_features = './NonLinCFA/temp_prec_snow_copernicus/'

basins_snow = ['Dora', 'Piemonte_Nord', 'Piemonte_Sud']

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins_snow:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train_withSnow.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val_withSnow.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test_withSnow.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

#### Temp Prec Snow Lakes

In [ ]:
### wrapper best 5 features
path_features = './NonLinCFA/temp_prec_snow_lakes/'

#best5_wrapper_fulldf_train = pd.DataFrame()
#best5_wrapper_fulldf_val = pd.DataFrame()
#best5_wrapper_fulldf_test = pd.DataFrame()
basins_lakes = ['Adda', 'Oglio_Iseo', 'Ticino']

for basin in basins_lakes:
    best5_wrapper_train_temp = pd.read_csv(path_features+basin+'_wrapper_best5_train_withSnowLakes.csv')
    best5_wrapper_val_temp = pd.read_csv(path_features+basin+'_wrapper_best5_val_withSnowLakes.csv')
    best5_wrapper_test_temp = pd.read_csv(path_features+basin+'_wrapper_best5_test_withSnowLakes.csv')
    best5_wrapper_fulldf_train[basin+'_'+best5_wrapper_train_temp.columns.values] = best5_wrapper_train_temp
    best5_wrapper_fulldf_val[basin+'_'+best5_wrapper_val_temp.columns.values] = best5_wrapper_val_temp
    best5_wrapper_fulldf_test[basin+'_'+best5_wrapper_test_temp.columns.values] = best5_wrapper_test_temp

In [ ]:
### CMI features
path_features = './NonLinCFA/temp_prec_snow_lakes/'

#CMI_fulldf_train = pd.DataFrame()
#CMI_fulldf_val = pd.DataFrame()
#CMI_fulldf_test = pd.DataFrame()

for basin in basins_lakes:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_train_withSnowLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_val_withSnowLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_test_withSnowLakes.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
basins = ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino']

### CMI best5 features
path_features = './NonLinCFA/temp_prec_snow_lakes_copernicus/'

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train_withSnowLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val_withSnowLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test_withSnowLakes.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

#### Temp Prec Lakes

In [ ]:
### wrapper best 5 features
path_features = './NonLinCFA/temp_prec_lakes/'

#best5_wrapper_fulldf_train = pd.DataFrame()
#best5_wrapper_fulldf_val = pd.DataFrame()
#best5_wrapper_fulldf_test = pd.DataFrame()

basins_lakes = ['Adda', 'Oglio_Iseo', 'Ticino']

for basin in basins_lakes:
    best5_wrapper_train_temp = pd.read_csv(path_features+basin+'_wrapper_best5_train_withLakes.csv')
    best5_wrapper_val_temp = pd.read_csv(path_features+basin+'_wrapper_best5_val_withLakes.csv')
    best5_wrapper_test_temp = pd.read_csv(path_features+basin+'_wrapper_best5_test_withLakes.csv')
    best5_wrapper_fulldf_train[basin+'_'+best5_wrapper_train_temp.columns.values] = best5_wrapper_train_temp
    best5_wrapper_fulldf_val[basin+'_'+best5_wrapper_val_temp.columns.values] = best5_wrapper_val_temp
    best5_wrapper_fulldf_test[basin+'_'+best5_wrapper_test_temp.columns.values] = best5_wrapper_test_temp

In [ ]:
### CMI features
path_features = './NonLinCFA/temp_prec_lakes/'

#CMI_fulldf_train = pd.DataFrame()
#CMI_fulldf_val = pd.DataFrame()
#CMI_fulldf_test = pd.DataFrame()

for basin in basins_lakes:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_train_withLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_val_withLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_test_withLakes.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
### CMI best5 features
path_features = './NonLinCFA/temp_prec_lakes/'

#best5_CMI_fulldf_train = pd.DataFrame()
#best5_CMI_fulldf_val = pd.DataFrame()
#best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins_lakes:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train_withLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val_withLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test_withLakes.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

### Wrapper

In [ ]:
MTL_scores(clust_basins=['Emiliani1', 'Emiliani2', 'Garda_Mincio'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
0.3110223474860595
Emiliani2
0.1956025095830476
Garda_Mincio
0.23649598607427524


In [ ]:
MTL_scores(clust_basins=['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Adda
-0.1593577323630866
Lambro_Olona
-0.14798724778122918
Oglio_Iseo
-0.12522732053038332
Ticino
-0.08833494197836544


In [ ]:
MTL_scores(clust_basins=['Dora', 'Piemonte_Nord', 'Piemonte_Sud'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
-0.5459713096478052
Piemonte_Nord
-0.40430308858467967
Piemonte_Sud
-0.450284076271398


In [ ]:
MTL_scores(clust_basins=['Piemonte_Nord', 'Piemonte_Sud'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Piemonte_Nord
0.16988107220330162
Piemonte_Sud
0.12037698240049333


### CMI

In [ ]:
MTL_scores(clust_basins=['Emiliani1', 'Emiliani2', 'Garda_Mincio'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
0.2190543851369916
Emiliani2
0.07044281068946567
Garda_Mincio
0.19610076223593975


In [ ]:
MTL_scores(clust_basins=['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Adda
-0.7465158379401895
Lambro_Olona
-0.8040102239972284
Oglio_Iseo
-0.72359826472957
Ticino
-0.6447302894178986


In [ ]:
MTL_scores(clust_basins=['Dora', 'Piemonte_Nord', 'Piemonte_Sud'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
-0.769870383079041
Piemonte_Nord
-0.5071374298289144
Piemonte_Sud
-0.5736278714662375


In [ ]:
MTL_scores(clust_basins=['Piemonte_Nord', 'Piemonte_Sud'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Piemonte_Nord
0.054008126502080755
Piemonte_Sud
-0.0371747221951928


### CMI best 5

In [ ]:
MTL_scores(clust_basins=['Emiliani1', 'Emiliani2', 'Garda_Mincio'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
0.3010416684220274
Emiliani2
0.2510014354078456
Garda_Mincio
0.15646547533555533


In [ ]:
MTL_scores(clust_basins=['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Adda
0.1389589955071342
Lambro_Olona
0.17464533572717544
Oglio_Iseo
0.1669436224333941
Ticino
0.17671234377443745


In [ ]:
MTL_scores(clust_basins=['Dora', 'Piemonte_Nord', 'Piemonte_Sud'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
-0.2749769463416645
Piemonte_Nord
0.10531401798041751
Piemonte_Sud
0.029939273666413135


In [ ]:
MTL_scores(clust_basins=['Piemonte_Nord', 'Piemonte_Sud'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Piemonte_Nord
0.054008126502080755
Piemonte_Sud
-0.0371747221951928
